In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but [a-zA-Z]+ was fitted with feature names",
    category=UserWarning,
)

warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

HTML(open("custom.html", "r").read())

# Chapter 6: An overview of classifiers, Part 2

<span style="font-size: 150%;">Decision trees, ensemble methods and summary</span>

Let's repeat our helper functions from previous part:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


def samples_color(ilabels, colors=["steelblue", "chocolate"]):
    """Return colors list from labels list given as indices."""
    return [colors[int(i)] for i in ilabels]


def plot_decision_surface(
    features_2d,
    labels,
    classifier,
    preprocessing=None,
    plt=plt,
    marker=".",
    N=100,
    alpha=0.2,
    colors=["steelblue", "chocolate"],
    title=None,
    test_features_2d=None,
    test_labels=None,
    test_s=60,
):
    """Plot a 2D decision surface for a already trained classifier."""

    # sanity check
    assert len(features_2d.columns) == 2

    # pandas to numpy array; get min/max values
    xy = np.array(features_2d)
    min_x, min_y = xy.min(axis=0)
    max_x, max_y = xy.max(axis=0)

    # create mesh of NxN points; tech: `N*1j` is spec for including max value
    XX, YY = np.mgrid[min_x : max_x : N * 1j, min_y : max_y : N * 1j]
    points = np.c_[XX.ravel(), YY.ravel()]  # shape: (N*N)x2
    points = pd.DataFrame(points, columns=["x", "y"])

    # apply scikit-learn API preprocessing
    if preprocessing is not None:
        points = preprocessing.transform(points)

    # classify grid points
    classes = classifier.predict(points)

    # plot classes color mesh
    ZZ = classes.reshape(XX.shape)  # shape: NxN
    plt.pcolormesh(
        XX,
        YY,
        ZZ,
        alpha=alpha,
        cmap=matplotlib.colors.ListedColormap(colors),
        shading="auto",
    )
    # plot points
    plt.scatter(
        xy[:, 0],
        xy[:, 1],
        marker=marker,
        color=samples_color(labels, colors=colors),
    )
    # set title
    if title:
        if hasattr(plt, "set_title"):
            plt.set_title(title)
        else:
            plt.title(title)
    # plot test points
    if test_features_2d is not None:
        assert test_labels is not None
        assert len(test_features_2d.columns) == 2
        test_xy = np.array(test_features_2d)
        plt.scatter(
            test_xy[:, 0],
            test_xy[:, 1],
            s=test_s,
            facecolors="none",
            linewidths=2,
            color=samples_color(test_labels),
        );

Since the latest version, `sklearn` offers its own method for visualizing decision boundaries: `DecisionBoundaryDisplay`. Documentation for this method can be found here: https://scikit-learn.org/stable/modules/generated/sklearn.inspection.DecisionBoundaryDisplay.html.

## Decision trees

Let's see what a decision tree is by looking at an (artificial) example: 

Following the yes/no decision nodes from the top (root node) you will end up at a **leaf** which concludes if you should work or relax: 

<table>
    <tr><td><img src="./images/decision_tree-work.png" width=600px></td></tr>
</table>

### How are the decision tree splits selected?

Starting from the top the decision tree is build by selecting **best split of the dataset using a single feature**. 

Such a split is done on a single feature and splits your space into two halves.

The optimal feature and its split value are determined for each node such that the resulting **subsets are as pure as possible** .

Purity of a set is measured using so-called [gini-index](https://en.wikipedia.org/wiki/Diversity_index#Gini%E2%80%93Simpson_index). The lower this index the more pure a set is.

The formula to compute this is index is $1 - p_0^2 - p_1^2$, where $p_i$ is the fraction of samples of class $i$ in the set. 

Thus for a set with only postive or negative samples either $p_0$ or $p_1$ is 1.0 and the other is 0, resulting in an optimal gini index of $0$.



<table>
    <tr><td><img src="./images/decision_tree-split.png" width=600px></td></tr>
</table>

In the left tree the gini index of the left split is $0$, and the one from the right split is $1 - 0.4^2 - 0.6^2 = 0.48$. The weighted sum (using the number of elements in each set as weights) is then $0.48 * 5/7 = 0.34$

An similar computation yields a value of $0.21$ for the right split, and thus the right split would be preferred when constructing the tree.


### XOR decision tree

Let's try out decision trees with the XOR dataset, in which samples have class `True` when the two coordinates `x` and `y` have different sign, otherwise they have class `False`.

In [ ]:
import pandas as pd


df = pd.read_csv("data/xor.csv")
features_2d = df.loc[:, ("x", "y")]
labelv = df["label"]

plt.figure(figsize=(5, 5))
plt.xlabel("x")
plt.ylabel("y")
plt.title("Orange is True, blue is False")
plt.scatter(features_2d.iloc[:, 0], features_2d.iloc[:, 1], color=samples_color(labelv));

Decision trees live in the `sklearn.tree` module.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


# Note: split randomness picked manually for educational purpose
X_train, X_test, y_train, y_test = train_test_split(
    features_2d, labelv, random_state=10
)

# Note: features are permuted reandomly in case equally good splits are found
# fix randomization for reproduciblity
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(X_train, y_train)

print("train score: {:.2f}%".format(100 * classifier.score(X_train, y_train)))
print("test score: {:.2f}%".format(100 * classifier.score(X_test, y_test)))

plt.figure(figsize=(5, 5))
plot_decision_surface(
    features_2d,
    labelv,
    classifier,
    test_features_2d=X_test,
    test_labels=y_test,
)

About the plot: **the points surrounded with a circle are from the test data set** (not used for learning), all other points belong to the training data.

This surface seems a bit rough on edges. One of the biggest advantages of the decision trees is interpretability of the model. Let's **inspect the model by looking at the tree that was built**:

In [ ]:
from sklearn.tree import plot_tree


fig = plt.figure(figsize=(12, 8))
fig.suptitle("XOR Decision Tree")
plot_tree(classifier, feature_names=["x", "y"], class_names=["False", "True"]);

<span style="font-size: 150%">Whoaaa .. what happened here?</span>

XOR is the **anti-example** for DTs!

They cannot make the "natural" split at value `0`:
Either splitting at $x=0$ or $y=0$ would leave very impure splits with ~50% of samples from both classes. Instead the tree focusses on details resp. noise in the particular data set during construction.

Note: This would be not an issue if a decision tree would not constructed in a greedy manner, split per split.


Moreover, the tree is quite deep because, by default, it is built until all nodes are "pure" (`gini = 0.0`). This tree is **overfitted**.

### How to avoid overfitting?

There is no regularization penalty like in logistic regression or SVM methods when bulding a decision tree. Instead we can set learning hyperparameters such as:
or
* upper limit on tree depth (`max_depth`), or
* a minimum number of samples required at a node or at a leaf node (`min_samples_split`, `min_samples_leaf`), or
* an early stopping criteria based on minumum value of impurity or on minimum decrease in impurity (`min_impurity_split`, `min_impurity_decrease`),
* tree pruning (based on minimal cost-complexity; `ccp_alpha`) after the tree has been built, 
* ... and few more - see `DecisionTreeClassifier` docs.


### Exercise section

1. In theory for the XOR dataset it should suffice to use each feature exactly once with splits at `0`, but the decision tree learning algorithm is unable to find such a solution. Play around with `max_depth` to get a smaller but similarly performing decision tree for the XOR dataset.<br/>
  Bonus question: which other hyperparameter could you have used to get the same result?

2. Build a decision tree for the beers dataset. Use maximum depth and tree pruning strategies to get a much smaller tree that performs as well as the default tree.<br/>
  Note: `classifier.tree_` instance has attributes such as `max_depth`, `node_count`, or `n_leaves`, which measure size of the tree.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree


df = pd.read_csv("data/xor.csv")
features_2d = df.loc[:, ("x", "y")]
labelv = df["label"]

max_depths = [2, 3, 4]
# ...

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree

df = pd.read_csv("data/beers.csv")
features = df.iloc[:, :-1]
labelv = df.iloc[:, -1]
# ...

In [ ]:
# SOLUTION 2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree


df = pd.read_csv("data/beers.csv")
print(df.head(2))

features = df.iloc[:, :-1]
labelv = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(features, labelv, random_state=10)

# default
classifier = DecisionTreeClassifier(random_state=0)
pipeline = make_pipeline(StandardScaler(), classifier)
pipeline.fit(X_train, y_train)
print()
print("#### default Beers Decision Tree")
print(
    f"depth: {classifier.tree_.max_depth}, ",
    f"#nodes: {classifier.tree_.node_count}, ",
    f"#leaves: {classifier.tree_.n_leaves}",
)
print(f"train score: {100 * pipeline.score(X_train, y_train):.2f}%")
print(f" test score: {100 * pipeline.score(X_test, y_test):.2f}%")

# smaller
classifier = DecisionTreeClassifier(max_depth=4, ccp_alpha=0.02, random_state=0)
pipeline = make_pipeline(StandardScaler(), classifier)
pipeline.fit(X_train, y_train)
print()
print("#### smaller Beers Decision Tree")
print(
    f"depth: {classifier.tree_.max_depth}, ",
    f"#nodes: {classifier.tree_.node_count}, ",
    f"#leaves: {classifier.tree_.n_leaves}",
)
print(f"train score: {100 * pipeline.score(X_train, y_train):.2f}%")
print(f" test score: {100 * pipeline.score(X_test, y_test):.2f}%")

fig = plt.figure(figsize=(10, 6))
plot_tree(classifier, feature_names=features.columns.values);

One **issue with decision trees is their instability** - a small changes in the training data usually results in a completely different order of splits (different tree structure).

## Ensemble Averaging: Random Forests

The idea of Random Forest method is to generate **ensemble of many "weak" decision trees** and by **averaging out their probabilistic predictions**. (The original Random Forests method used voting.)


Weak classifier here are **shallow trees with feature-splits picked only out of random subsets of features** (*features bagging*). Random subset of features is selected per each split, not for the whole classifier.

<table>
    <tr><td><img src="./images/random_forest.png" width=800px></td></tr>
    <tr><td><center><sub>Source: <a href="https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249">https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249</a></sub></center></td></tr>
</table>


### Demonstration

You will find Random Forest method implementation in the `sklearn.ensemble` module.

The main parameters are:
* number of trees (`n_estimators`),
* each tree max. depth 2 (`max_depth`), and
* max. number of randomly selected features to pick from when building each tree (`max_features`).

Let's build a small Random Forest and have a look at its trees, available under `.estimators_` property.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import plot_tree


df = pd.read_csv("data/beers.csv")
print(df.head(2))

features = df.iloc[:, :-1]
labelv = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(features, labelv, random_state=10)

# 4 shallow (depth 2) trees, each using only 3 randomly selected features
# total: up to 4*3 decision nodes, up to 4*4 class nodes
n_trees = 4
classifier = RandomForestClassifier(
    max_depth=2,
    n_estimators=n_trees,
    max_features=3,
    random_state=0,
)
pipeline = make_pipeline(StandardScaler(), classifier)
pipeline.fit(X_train, y_train)

print()
print("#### Random Forest")
print(f"train score: {100 * pipeline.score(X_train, y_train):.2f}%")
print(f" test score: {100 * pipeline.score(X_test, y_test):.2f}%")

# to evaluate ensemble estimators, we need to use transformed data
X_train_trans = pipeline[:-1].transform(X_train)
X_test_trans = pipeline[:-1].transform(X_test)

fig, ax_arr = plt.subplots(ncols=n_trees, nrows=1, figsize=(7 * n_trees, 5))
for i, internal_classifier in enumerate(classifier.estimators_):
    ax = ax_arr[i]
    plot_tree(internal_classifier, feature_names=features.columns.values, ax=ax)
    ax.set_title(
        (
            f"Tree #{i}\n"
            f"train score: {100 * internal_classifier.score(X_train_trans, y_train):.2f}%\n"
            f" test score: {100 * internal_classifier.score(X_test_trans, y_test):.2f}%"
        )
    )

Random forests are fast and shine with high dimensional data (many features).

<div class="alert alert-block alert-info">
    <p><i class="fa fa-info-circle"></i>
        Random Forest can estimate <em>out-of-bag error</em> (OOB) while learning; set <code>oob_score=True</code>. (The out-of-bag (OOB) error is the average error for each data sample, calculated using predictions from the trees that do not contain that sample in their respective bootstrap samples.)
    OOB is a generalisation/predictive error that, together with <code>warm_start=True</code>, can be used for efficient search for a good-enough number of trees, i.e. the <code>n_estimators</code> hyperparameter value (see: <a href=https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html>OOB Errors for Random Forests</a>).
    </p>
</div>

## Boosting: AdaBoost

<span style="font-size: 125%;">What is it?</span>

Boosting is another sub-type of ensemble learning. Same as in averaging, the idea is to generate many **weak classifiers to create a single strong classifier**, but in contrast to averaging, the classifiers are learnt **iteratively**.

<span style="font-size: 125%;">How does it work?</span>

Each iteration focuses more on **previously misclassified samples**. To that end, **data samples are weighted**, and after each learning iteration the data weights are readjusted.

<table>
    <tr><td><img src="./images/AdaBoost.png" width=800px></td></tr>
    <tr><td><center><sub>Source: Marsh, B., (2016), <em>Multivariate Analysis of the Vector Boson Fusion Higgs Boson</em>.</sub></center></td></tr>
</table>

The final prediction is a weighted majority vote or weighted sum of predictions of the weighted weak classifiers.

Boosting works very well out of the box. There is usually no need to fine tune method hyperparameters to get good performance.

<span style="font-size: 125%;">Where do i start?</span>

**AdaBoost (“Adaptive Boosting”) is a baseline boosting algorithm** that originally used decisoin trees as weak classifiers, but, in principle, works with any classification method (`base_estimator` parameter).

In each AdaBoost learning iteration, additionally to samples weights, the **weak classifiers are weighted**. Their weights are readjusted, such that **the more accurate a weak classifier is, the larger its weight is**.


### Demonstration

You will find AdaBoost algorithm implementation in the `sklearn.ensemble` module.

We'll use `n_estimators` parameter to determine number of weak classifiers. These by default are single node decision trees (`base_estimator = DecisionTreeClassifier(max_depth=1)`). We can examine them via `.estimators_` property of a trained method.

For presentation, in order to weight the classifiers, we will use the original discrete AdaBoost learning method (`algorithm="SAMME"`). Because the classifiers learn iteratively on differently weighted samples, to understand the weights we have to look at internal train errors and not at the final scores on the training data.

In [ ]:
from math import ceil, floor

import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import plot_tree


df = pd.read_csv("data/beers.csv")
print(df.head(2))

features = df.iloc[:, :-1]
labelv = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(features, labelv, random_state=10)

# 9 single node decision trees
# total: 9*1 decision nodes, 9*2 class nodes
# (Note: with default real AdaBoost "SAMME.R" algorithm all weights are 1 at the end)
n_trees = 9
classifier = AdaBoostClassifier(n_estimators=n_trees, algorithm="SAMME", random_state=0)
pipeline = make_pipeline(StandardScaler(), classifier)
pipeline.fit(X_train, y_train)

print()
print("AdaBoost")
print(f"train score: {100 * pipeline.score(X_train, y_train):.2f}%")
print(f"test score: {100 * pipeline.score(X_test, y_test):.2f}%")

# to evaluate ensemble estimators, we need to use transformed data
X_train_trans = pipeline[:-1].transform(X_train)
X_test_trans = pipeline[:-1].transform(X_test)

fig, ax_arr = plt.subplots(ncols=n_trees, nrows=1, figsize=(5 * n_trees, 4))
for i, internal_classifier in enumerate(classifier.estimators_):
    ax = ax_arr[i]
    plot_tree(internal_classifier, feature_names=features.columns.values, ax=ax)
    ax.set_title(
        (
            f"Tree #{i}, weight: {classifier.estimator_weights_[i]:.2f}\n"
            f"train error: {classifier.estimator_errors_[i]:.2f}\n"
            f"(train score: {100 * internal_classifier.score(X_train_trans, y_train):.2f}%)\n"
            f"test score: {100 * internal_classifier.score(X_test_trans, y_test):.2f}%"
        )
    )

### Other boosting methods

In practice you will mostly want to use other than AdaBoost methods for boosting.

####  Gradient Tree Boosting (GTB)

It re-formulates boosting problem as an optimization problem which is solved with efficient Stochastic Gradient Descent optimization method (more on that in the neuronal networks script).

In contrast to AdaBoost, GTB relies on using decision trees.

In particular, try out [XGboost](https://xgboost.readthedocs.io/en/latest/); it's a package that won many competitions, cf. [XGboost@Kaggle](https://www.kaggle.com/dansbecker/xgboost). It is not part of scikit-learn, but it offers a `scikit-learn` API (see https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn ); a `scikit-learn` equivalent is [`GradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

#### Histogram-based Gradient Boosting Classification Tree.

A new `scikit-learn` implementation of boosting based on decision trees is [`HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html). It is much faster then `GradientBoostingClassifier` for big datasets (`n_samples >= 10 000`).




## Ensemble Stacking: a honorary mention

Stacking is used often in case of different types of base models, when it's not clear which type of model will perform best.

The stacking architectore combines multiple classfiers which feed into a final meta-model (called also *combiner*, *blender*, or *generalizer*). Here is an example:


<table>
    <tr><td><img src="./images/ensemble-learning-stacking.png" width="400px"></td></tr>
    <tr><td><center><sub><a href="https://data-science-blog.com/blog/2017/12/03/ensemble-learning/">https://data-science-blog.com/blog/2017/12/03/ensemble-learning/</a></sub></center></td></tr>
</table>

The learning procedure is as follows:
- split your training data (e.g. in an 80-20 split),
- train the first layer on the 80% of the data,
- feed the remaining 20% through the first layer,
- train the meta-model on the output from these 20%.

The idea is that the meta-model also is able to correct overfitting from the first layer.

Stacking combines strengths of different models and usually slightly outperforms the best individual model. In practice often multiple stacking layers are used with groups of different but repeating types of classifiers.

<table>
    <tr><td><center><img src="./images/ensemble-learning-stacking-kdd_2015_winner.png" width="800px"></center></td></tr>
    <tr><td><center><sub>KDD Cup 2015 winner</sub></center></td></tr>
    <tr><td><center><sub>GBM: Gradient Boosting Machine; NN: Neural Network; FM: Factorization Machine; LR: Logistic Regression; KRR: Kernel Ridge Regression; ET: Extra Trees; RF: Random Forests; KNN: K-Nearest Neighbors</sub></center></td></tr>
    <tr><td><center><sub><a href="https://www.slideshare.net/jeongyoonlee/winning-data-science-competitions-74391113"> Jeong-Yoon Lee, <em>Winning Data Science Competitions</em>, Apr 2017</a></sub></center></td></tr>
</table>

In the `sklearn.ensemble` the stacking is implemented by `StackingClassifier` and `StackingRegressor`.

## Why does ensemble learning work?

* Probability of making an error by majority of the classifiers in the ensemble is much lower then error that each of the weak classifiers makes alone.

* An ensemble classifier is more roboust (has lower variance) with respect to the training data.

* The weak classifiers are small, fast to learn, and, in case of averaging, they can be learnt in parallel.

In general, **usually ensemble classifier performs better than any of the weak classifiers in the ensemble**.

## Coding session

Apply all classifiers seen so far to the MNIST images dataset (`from sklearn.datasets import load_digits`). To that end:

1. split your data into training and a test "holdout" data (`from sklearn.model_selection import train_test_split`) in 80:20 proporition; stratify the 10 target classes (`stratify=...`);
2. compare cross validation mean f1 scores; use the stratified k-fold CV strategy (`from sklearn.model_selection import cross_val_score, StratifiedKFold`; attention: this is a non-binary multi-class problem, you will have to use an adjusted f1 score, e.g. unweighted per class mean via `scoring="f1_macro"` keyword arg - see [the `scoring` parameter predefined values](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules));
3. train the final model on the whole dataset and, use the test "holdout" data to repot a final model f1 performance; report also accuracy, precision and recall scores (`from sklearn.metrics import classification_report`);
4. next, try to manually tune hyperparameters to minimize the train test gap and to squeeze out at least 90% cross validation f1 score performance out of each classifier; try using PCA preprocessing (`sklearn.pipeline.make_pipeline`, `sklearn.decomposition.PCA`); what about data scaling? which models are most effective and easiest to tune manually?
5. optionally, once you get a feel of good preprocessing and classifier hyperparameters, define parameters ranges and apply a CV-based search to find optimal values for the hyperparameters (`from sklearn.model_selection import GridSearchCV, RandomizedSearchCV`).

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA

# classifiers
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier


digits = load_digits()

labels = digits.target
n_samples = len(labels)

# flatten images of shape N_SAMPLES x 8 x 8 to N_SAMPLES x 64:
print("digits data set shape:", digits.images.shape)
features = digits.images.reshape((n_samples, -1))
print("feature matrix shape:", features.shape)

# (
#     features_train,
#     features_test,
#     labels_train,
#     labels_test,
# ) = train_test_split(..., test_size=..., stratify=..., random_state=42)
#
# classifiers = [
#     LogisticRegression(...),
#     LinearSVC(...),
#     SVC(...),
#     DecisionTreeClassifier(..., random_state=42),  # rather won't do very well when not used in an ensemble method
#     RandomForestClassifier(..., random_state=42),
#     AdaBoostClassifier(..., random_state=42),
# ]
#
# pipeline = make_pipeline(...)
#
# cross_validator = StratifiedKFold(...)
# scores = cross_val_score(..., scoring=..., cv=...)
# ...

## Summary

Below you will find a table with some guidelines, as well as pros and cons of different classication methods available in scikit-learn.

<div class="alert alert-block alert-warning">
    <p><i class="fa fa-warning"></i>&nbsp;<strong>Summary table</strong></p>

<p>
<em>Disclaimer</em>: this table is neither a single source of truth nor complete - it's intended only to provide some first considerations when starting out. At the end of the day, you have to try and pick a method that works for your problem/data.
</p>

<table>
<thead>
<tr>
<th style="text-align: center;">Classifier type</th>
<th style="text-align: center;">When?</th>
<th style="text-align: center;">Advantages</th>
<th style="text-align: center;">Disadvantages</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align: left;">Nearest Neighbors<br><br><code>KNeighborsClassifier</code></td>
<td style="text-align: left;">- numeric data<br> - when (fast) linear classifiers do not work</td>
<td style="text-align: left;">- simple (not many parameters to tweak), hence, a good baseline classifier</td>
<td style="text-align: left;">- known not to work well for many dimensions (20 or even less features)</td>
</tr>
<tr>
<td style="text-align: left;">Logistic Regression<br><br><code>LogisticRegression</code></td>
<td style="text-align: left;">- high-dimensional data<br> - a lot of data</td>
<td style="text-align: left;">- fast, also in high dimensions<br> - weights can be interpreted</td>
<td style="text-align: left;">- data has to be linearly separable (happens often in higher dimensions)<br> - not very efficient with large number of samples</td>
</tr>
<tr>
<td style="text-align: left;">Linear SVM<br><br><code>LinearSVC</code></td>
<td style="text-align: left;">same as above but might be better for text analysis (many features)</td>
<td style="text-align: left;">same as above but might be better with very large number of features</td>
<td style="text-align: left;">same as above but possibly a bit better with large number of samples</td>
</tr>
<tr>
<td style="text-align: left;">Kernel SVM<br><br><code>SVC</code></td>
<td style="text-align: left;">same as above but when linear SVM does not work<br>- not too many data points</td>
<td style="text-align: left;">same as above but learns non-linear boundaries</td>
<td style="text-align: left;">same as above but much slower and requires data scaling<br>- model is not easily interpretable</td>
</tr>
<tr>
<td style="text-align: left;">Decision Tree<br><br><code>DecisionTreeClassifier</code></td>
<td style="text-align: left;">- for illustration/insight<br> - with multi-class problems <br> - with categorical or mixed categorical and numerical data</td>
<td style="text-align: left;">- simple to interpret<br> - good classification speed and performance</td>
<td style="text-align: left;">- prone to overfitting<br> - unstable: small change in the training data can give very different model</td>
</tr>
<tr>
<td style="text-align: left;">Ensemble Averaging<br><br><code>RandomForestClassifier</code></td>
<td style="text-align: left;">- when decision tree would be used but for performance</td>
<td style="text-align: left;">- fixes decision tree issues: does not overfit easily and is stable with respect to training data<br> - takes into account features dependencies<br> - can compute predicitve error when learning<br> ...</td>
<td style="text-align: left;">- harder to interpret than a single decision tree</td>
</tr>
<tr>
<td style="text-align: left;">Boosting<br><br><code>AdaBoostClassifier</code> (<code>XGBClassifier</code>, <code>HistGradientBoostingClassifier</code>)</td>
<td style="text-align: left;">same as above</td>
<td style="text-align: left;">- works very well out-of-the-box<br>- better performance and more interpretable than random forest when using depth 1 trees</td>
<td style="text-align: left;">- more prone to overfitting than random forest</td>
</tr>
<tr>
<td style="text-align: left;">Stacking<br><br><code>StackingClassifier</code></td>
<td style="text-align: left;">- when having multiple various learners (with different weaknesses)<br>- when not having enough data to use neuronal networks</td>
<td style="text-align: left;">- works well out-of-the-box<br>- improves performance of even already good learners</td>
<td style="text-align: left;">- complicates interpretability of results<br>- takes time to train and to build a multi-layer architecture (if enough data, it's easier to use neuronal networks)</td>
</tr>
<tr style="border-bottom:1px solid black">
    <td colspan="100%"></td>
</tr>
<tr>
<td colspan="100%" style="text-align: center;"><em>[not shown here]</em></td>
</tr>
<tr>
<td style="text-align: left;">Naive Bayes<br><br><code>ComplementNB</code>, ...</td>
<td style="text-align: left;">- with text data</td>
<td style="text-align: left;">...</td>
<td style="text-align: left;">...</td>
</tr>
<tr>
<td style="text-align: left;">Stochastic Gradient<br><br><code>SGDClassifier</code></td>
<td style="text-align: left;">- with really big data</td>
<td style="text-align: left;">...</td>
<td style="text-align: left;">...</td>
</tr>
<tr>
<td style="text-align: left;">Kernel Approximation<br><br>pipeline: <code>RBFSampler</code> or <code>Nystroem</code> + <code>LinearSVC</code></td>
<td style="text-align: left;">- with really big data and on-line training</td>
<td style="text-align: left;">...</td>
<td style="text-align: left;">...</td>
</tr>
</tbody>
</table>

</div>

You should be able now to understand better the classification part of the ["Choosing the right estimator" scikit-learn chart ](https://scikit-learn.org/stable/tutorial/machine_learning_map/):


<table>
    <tr><td><img src="./images/scikit-learn_ml_map-classification.png" width=800px></td></tr>
    <tr><td><center><sub>Source: <a href="https://scikit-learn.org/stable/tutorial/machine_learning_map/">https://scikit-learn.org/stable/tutorial/machine_learning_map/</a></sub></center></td></tr>
</table>

Copyright (C) 2019-2022 ETH Zurich, SIS ID